In [5]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch.functional import F

In [6]:
# HYPERPARAMS
FILTER_SIZE_1 = 3     # Convolutional Layer 1
FILTER_NUM_1 = 32
FILTER_SIZE_2 = 3     # Convolutional Layer 2
FILTER_NUM_2 = 32
FILTER_SIZE_3 = 3     # Convolutional Layer 3
FILTER_NUM_3 = 64
FC_SIZE = 128         # Number of neurons in fully-connected layer
H_IMG, W_IMG, C_IMG = 180, 270, 3
CLASSES = ['apple', 'carambola', 'pear', 'plum', 'tomato']
NUM_CLASSES = len(CLASSES)
BATCH_SIZE = 16
DATASET_SIZE = 12005
SPLIT_PROPORTION = 0.2
CHECKPOINT_DIR = "../models/"
EPOCHS = 32
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 5e-4
VALID_DATA_PATH = "../Attachment/Attachment 3/"
DATA_PATH = "../Attachment/Attachment 2/"

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(C_IMG, FILTER_NUM_1, FILTER_SIZE_1, padding=FILTER_SIZE_1//2)
        self.conv2 = nn.Conv2d(FILTER_NUM_1, FILTER_NUM_2, FILTER_SIZE_2, padding=FILTER_SIZE_2//2)
        self.conv3 = nn.Conv2d(FILTER_NUM_2, FILTER_NUM_3, FILTER_SIZE_3, padding=FILTER_SIZE_3//2)

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers
        # Calculate the size of the flattened layer
        self.fc1_size = self._get_conv_output((C_IMG, H_IMG, W_IMG))
        self.fc1 = nn.Linear(self.fc1_size, FC_SIZE)
        self.fc2 = nn.Linear(FC_SIZE, NUM_CLASSES)

    def forward(self, x):
        x = x.float()
        
        # Reorder to [BATCH_SIZE, channels, height, width]
        x = x.permute(0, 3, 1, 2)  
        
        # Apply convolutions and max pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the output for the dense layer
        x = x.reshape(-1, self.fc1_size)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def _get_conv_output(self, shape):
        with torch.no_grad():
            input = torch.rand(1, *shape)
            output = self._forward_features(input)
            return int(np.prod(output.size()))

    def _forward_features(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        return x

model = ConvNet()

In [10]:
# Load checkpoint
checkpoint = torch.load('../models/checkpoint.pt',map_location=torch.device('cpu'))
# Load model state
model.load_state_dict(checkpoint['model_state_dict'])
# Load optimizer state (if saved)
if 'optimizer_state_dict' in checkpoint:
    print(True)
    #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# If you also saved other items like the epoch, loss, etc.
if 'epoch' in checkpoint:
    epoch = checkpoint['epoch']
if 'loss' in checkpoint:
    loss = checkpoint['loss']

# Don't forget to set the model to training or evaluation mode as needed
model.eval()  # or model.train(), depending on what you need


TypeError: 'ConvNet' object is not subscriptable

In [ ]:
VALID_DATA_PATH = "../Attachment/Attachment 3/"
valid_data_x = []
valid_data_y = []
ids = []
for path in os.listdir(VALID_DATA_PATH):
    image = cv2.imread(VALID_DATA_PATH + path)
    id = int(path.split('(')[1].split(')')[0])
    ids.append(id)
    valid_data_x.append(image)
    valid_data_y.append(0)
output_valid_x = valid_data_x
valid_data_x = np.array(valid_data_x) / 255.0
print(valid_data_x.shape)
valid_data_x = torch.tensor(np.array(valid_data_x))        
valid_data_y = torch.tensor(np.array(valid_data_y))        
valid_dataset = TensorDataset(valid_data_x, valid_data_y)
data_loader_valid = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)